---
# 신경망 종합 테스트
- burst 하여 active cell 이 많으면 새로운 패턴을 인지한다는 의미
- 먼저 spatial pooler 가 안정되어야 한다.
---
encoder - sp - tm -> prediction + textual representation<br/>
예측 cell / 활성화 cell 비교 - 예측이 맞는지 확인 가능<br/>
예측 column / 입력 column 비교 - 마찬가지<br/>
burst 한 column / 활성화 된 column - 마찬가지<br/>

---
Neural Network Comprehensive Test

If there is a lot of active cells by burst, it means to recognize new pattern

First, the spatial pooler must be stable. 

---
encoder - sp - tm -> prediction + textual representation

Compare predicted cell / activated cell - Can check whether prediction is correct

Prediction column / input column comparison - similar

burst column / active column - same


In [26]:
import numpy as np
import random
import nbimporter
import TM
import Encoder
import SP3

In [27]:
encoder = Encoder.ScalarEncoder(out_size=100, w=21, max_val=30)
sp = SP3.SpatialPooler(encoder.out_size, columns=512, perm_conn=.6, minOver=encoder.w/5, potential_rate=0.9)

In [28]:
for epoch in range(3):
    a = np.arange(encoder.max_val)
    random.shuffle(a)
    
    for i in a:
        input_data = encoder.encode(i)
        sp.compute_SDR(input_data)

In [29]:
for i in range(encoder.max_val):
    sp.compute_SDR(encoder.encode(i), learn=False)
    sdr1 = sp.getActiveColumns()
    sp.compute_SDR(encoder.encode(i+1), learn=False)
    sdr2 = sp.getActiveColumns()
    
    sp.similarity(sdr1, sdr2, False)
    print('')

유사도 : 0.45454545454545453
[  3  39  40 129 178 232 264 266 357 398 449]
[  3   4  39  40 129 209 357 400 430 507]

유사도 : 0.3
[  3   4  39  40 129 209 357 400 430 507]
[  4  39  41  90 137 175 209 233 241 327]

유사도 : 0.5
[  4  39  41  90 137 175 209 233 241 327]
[  4  22  41 173 175 233 250 327 354 467]

유사도 : 0.5
[  4  22  41 173 175 233 250 327 354 467]
[ 36  41  45 124 143 156 173 175 250 354]

유사도 : 0.2
[ 36  41  45 124 143 156 173 175 250 354]
[  0  24  45  51 123 136 341 346 354 373 374 500]

유사도 : 0.4166666666666667
[  0  24  45  51 123 136 341 346 354 373 374 500]
[ 45  51  99 123 176 205 212 255 278 341 346 361]

유사도 : 0.4166666666666667
[ 45  51  99 123 176 205 212 255 278 341 346 361]
[ 46 100 123 212 278 346 361 366 374 500]

유사도 : 0.4
[ 46 100 123 212 278 346 361 366 374 500]
[ 46  96  99 186 212 300 361 425 445 500]

유사도 : 0.1
[ 46  96  99 186 212 300 361 425 445 500]
[ 13  32  51  83  96 130 145 215 216 228 373 487]

유사도 : 0.4166666666666667
[ 13  32  51  83  96 130 145 2

---
# 테스트
- 정수 1, 2, 3, 4, 5 순서 학습
- burst 가 적게 되면 안정적으로 학습

- 이후에 1, 2, 4, 3, 5 처럼 데이터를 조작하여 학습
---
---
Test Integer
-1, 2, 3, 4, 5 order learning 
-When the burst is small, stable learning 
-Then, by manipulating the data like 1, 2, 4, 3, 5

In [30]:
tm = TM.TemporalMemory(sdr_size=sp.columnCount, max_seg=5, threshold=7)

In [31]:
tm.learn = True

seq_data = [1,2,3,4,5]

for i in range(10):
    for s in seq_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()        
        tm.compute(sdr)

In [32]:
tm.learn = False
prev_pred = 0 #is this breaking anything?
# 검증 시 학습기능 off On-the-fly learning function off
for d in seq_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)
    tm.test(prev_pred, sp.getActiveColumns())
    prev_pred = tm.column_predicted()
    
    print("현재 columns (sdr) : {}".format(sp.getActiveColumns()))
    print("다음 columns (예측Predict) : {}".format(tm.column_predicted()))
    print("현재 burst : {}".format(tm.column_bursted()))
    print("\n")
    

Loading BokehJS ...

예측 : 0.0
현재 columns (sdr) : [  3   4  39  40 129 209 357 400 430 507]
다음 columns (예측Predict) : [4, 327, 233, 41, 209, 241, 90]
현재 burst : [3, 4, 39]




Loading BokehJS ...

예측 : 0.7
현재 columns (sdr) : [  4  39  41  90 137 175 209 233 241 327]
다음 columns (예측Predict) : [354, 327, 41, 233, 173, 175, 467, 22, 250]
현재 burst : [137, 175, 39]




Loading BokehJS ...

예측 : 0.9
현재 columns (sdr) : [  4  22  41 173 175 233 250 327 354 467]
다음 columns (예측Predict) : [354, 36, 41, 173, 45, 143, 250]
현재 burst : [4]




Loading BokehJS ...

예측 : 0.7
현재 columns (sdr) : [ 36  41  45 124 143 156 173 175 250 354]
다음 columns (예측Predict) : [0, 354, 136, 45, 51, 500, 341, 374, 373, 24, 346, 123]
현재 burst : [156, 124, 175]




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [  0  24  45  51 123 136 341 346 354 373 374 500]
다음 columns (예측Predict) : [129, 3, 357, 39, 40, 430, 400, 209, 507]
현재 burst : [354, 374]




---
# 데이터 변경 Change Data

- 새로운 시퀀스를 학습
- 이전과 변화가 생긴 부분 (3->4, 4->5) 에서 예측 column 이 대략 2 배 되었는데 이전의 배웠던 시퀀스까지 고려하는 것임
- Temporal Memory 의 Segment 제한, synapse 제한을 조절하면 이전에 학습한 시퀀스의 망각(Segment, synapse 파괴) 시점을 조절할 수 있다.

---

- learning a new sequence
- In predicting and changing parts (3-> 4, 4-> 5), the predicted column is approximately doubled, considering the previous learned sequence.
- By controlling the segment limit and synapse limit of Temporal Memory, it is possible to control the timing of forgetting (Segment, synapse destruction) of the previously learned sequence.
---

In [33]:
tm.learn = True

test_data = [1,2,4,3,5]

for i in range(10):
    for s in test_data:
        dat = encoder.encode(s)
        sp.compute_SDR(dat, learn=False)
        sdr = sp.getActiveColumns_raw()
        tm.compute(sdr)

In [25]:
tm.learn = False

# 검증 시 학습기능 off
for d in test_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)    
    tm.test(prev_pred, sp.getActiveColumns())    
    prev_pred = tm.column_predicted()
    
    print("현재Now columns (sdr) : {}".format(sp.getActiveColumns()))
    print("다음Next columns (예측 Prediction) : {}".format(tm.column_predicted()))
    print("현재 burst : {}".format(tm.column_bursted()))
    print("\n")
    

test


Loading BokehJS ...

예측 : 1.0
현재Now columns (sdr) : [ 13  31  33 152 200 285 301 361 443 501]
다음Next columns (예측 Prediction) : [33, 388, 38, 490, 396, 301, 366, 81, 274, 51, 179, 501, 152, 154]
현재 burst : []


test


Loading BokehJS ...

예측 : 1.0
현재Now columns (sdr) : [ 33  38  51  81 152 154 179 274 301 366 388 396 490 501]
다음Next columns (예측 Prediction) : [388, 396, 148, 154, 416, 162, 36, 38, 174, 308, 53, 76, 344, 355, 363, 366, 369, 117, 126, 507, 254]
현재 burst : []


test


...

예측 : 1.0
현재Now columns (sdr) : [ 36  76 117 148 162 174 254 344 355 363 366 369 388]
다음Next columns (예측 Prediction) : [388, 139, 278, 154, 28, 287, 416, 38, 180, 308, 53, 184, 449, 203, 344, 355, 363, 366, 111, 117, 507, 126]
현재 burst : [162]


test


Loading BokehJS ...

예측 : 0.9230769230769231
현재Now columns (sdr) : [ 38  53 117 126 154 308 355 363 366 388 396 416 507]
다음Next columns (예측 Prediction) : [388, 139, 148, 278, 28, 287, 162, 36, 174, 180, 184, 449, 203, 76, 344, 355, 363, 366, 111, 369, 117, 254]
현재 burst : [363, 396]


test


Loading BokehJS ...

예측 : 1.0
현재Now columns (sdr) : [ 28 111 139 180 184 203 278 287 344 449]
다음Next columns (예측 Prediction) : [33, 200, 361, 13, 301, 501, 152, 443, 285, 31]
현재 burst : []




In [18]:
tm.learn = False
test_data = [1,2,4,3,5]
# 검증 시 학습기능 off
for d in test_data:
    dat = encoder.encode(d)
    sp.compute_SDR(dat, learn=False)
    sdr = sp.getActiveColumns_raw()
    tm.compute(sdr)
    tm.test(prev_pred, sp.getActiveColumns())
    prev_pred = tm.column_predicted()
    
    print("현재 columns (sdr) : {}".format(sp.getActiveColumns()))
    print("다음 columns (예측) : {}".format(tm.column_predicted()))
    print("현재 burst : {}".format(tm.column_bursted()))
    print("\n")
    

Loading BokehJS ...

예측 : 0.9
현재 columns (sdr) : [ 13  31  33 152 200 285 301 361 443 501]
다음 columns (예측) : [33, 388, 38, 490, 396, 301, 366, 81, 274, 51, 179, 501, 152, 154]
현재 burst : [152, 33, 443]




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 33  38  51  81 152 154 179 274 301 366 388 396 490 501]
다음 columns (예측) : [388, 396, 148, 154, 416, 162, 36, 38, 174, 308, 53, 76, 344, 355, 363, 366, 369, 117, 126, 507, 254]
현재 burst : []




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 36  76 117 148 162 174 254 344 355 363 366 369 388]
다음 columns (예측) : [388, 139, 396, 278, 154, 28, 287, 416, 38, 180, 308, 53, 184, 449, 203, 344, 355, 363, 366, 111, 117, 507, 126]
현재 burst : []




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 38  53 117 126 154 308 355 363 366 388 396 416 507]
다음 columns (예측) : [388, 139, 148, 278, 28, 287, 162, 36, 180, 184, 449, 203, 76, 344, 355, 363, 366, 111, 369, 117, 254]
현재 burst : [363]




Loading BokehJS ...

예측 : 1.0
현재 columns (sdr) : [ 28 111 139 180 184 203 278 287 344 449]
다음 columns (예측) : [200, 361, 13, 301, 501, 152, 443, 285, 31]
현재 burst : []




In [20]:
# len(tm.connInfo.totalSegments)
cell = 2000
print('{} 번 cell 에 연결되어 있는 synapse : {}'.format(cell, tm.connInfo.totalCells[cell].connectedSynapses))
#print(len(tm.prev_activeCells))
#print(len(tm.activeCells))
#for c in tm.connInfo.totalCells:
 #   if(len(c.connectedSynapses) > 0):
  #      print("살아있는 cell : " + str(c.cell_id))
        
#choose = list(tm.connInfo.totalCells[cell].connectedSynapses)[0]
#for i in tm.connInfo.totalSegments:
 #   for j in i.synapses:
  #      if (j.synapse_id == choose):
   #         print(j.synapse_id)
    #        print(j.preSynapticCell)
     #       print(j.permanence)
        
print("active Columns{}".format(sp.getActiveColumns()))
print("predicted Values:{}".format(tm.predicted))
print("bursted?{}".format(tm.bursted))

2000 번 cell 에 연결되어 있는 synapse : set()
active Columns[ 28 111 139 180 184 203 278 287 344 449]
predicted Values:{200, 361, 13, 301, 501, 152, 443, 285, 31}
bursted?set()
